# Fine-tune BLIP using Hugging Face `transformers`, `datasets`, `peft` 🤗 and `bitsandbytes`

Let's leverage recent advances from Parameter Efficient Fine-Tuning methods to fine-tune a large image to text model! We will show through this tutorial that it is possible to fine-tune a 3B scale model (~6GB in half-precision)

Here we will use a dummy dataset of [football players](https://huggingface.co/datasets/ybelkada/football-dataset) ⚽ that is uploaded on the Hub. The images have been manually selected together with the captions. 
Check the 🤗 [documentation](https://huggingface.co/docs/datasets/image_dataset) on how to create and upload your own image-text dataset.

## Set-up environment

## Load the image captioning dataset

Let's load the image captioning dataset, you just need few lines of code for that.

In [1]:
from datasets import load_dataset 
from PIL import Image
import pandas as pd
import numpy as np
import torch
import random
from transformers import AutoProcessor, Blip2ForConditionalGeneration
from time import time
random.seed(123)

#dataset = load_dataset("ybelkada/football-dataset", split="train")
embds_path = './train_preprocessed_embds'
df = pd.read_csv('full_train_annotations.csv') 

random_state = 42

# Pegue 80% dos dados como a primeira amostra
df_train = df.sample(frac=0.8, random_state=random_state)

# Pegue os 20% restantes como a segunda amostra
df_test = df.drop(df_train.index)

/home/jovyan/envs/lung-multimodal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's retrieve the caption of the first example:

## Create PyTorch Dataset

Let's define below the dataset as well as the data collator!

In [2]:
from torch.utils.data import Dataset, DataLoader


class ImageCaptioningDataset(Dataset):
    def __init__(self, df, embds_path, processor):
        self.dataset = df
        self.processor = processor
        self.embds_path = embds_path

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset.iloc[idx]
        image = np.load(f'{self.embds_path}/{item["NameinCTRATE"]}.npy')
        #print(image_rgb.min(), image_rgb.max())
        #encoding = self.processor(images=image, padding="max_length", return_tensors="pt")
        # remove batch dimension
        #encoding = {k: v.squeeze() for k, v in encoding.items()}
        #encoding["text"] = item["text"]
        #text_inputs = self.processor.tokenizer(item['report_text'] ,return_tensors="pt")
        return {'embds': torch.from_numpy(image), 'text': item['report_text']}

def collate_fn(batch):
    # pad the input_ids and attention_mask
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer([example["text"] for example in batch], padding=True, return_tensors="pt")
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
            # processed_batch["original_text"] = [example["text"] for example in batch]
            
    return processed_batch


In [3]:
class VisionModelOutput:
    def __init__(self, last_hidden_state):
        self.last_hidden_state = last_hidden_state

# Adaptador para ajustar a saída do ViT3D para o Q-Former
class ViT3DAdapter(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.training = True

    def forward(self, pixel_values=None, **kwargs):
        if self.training:
            return pixel_values.unsqueeze(0)
        else:
            return VisionModelOutput(last_hidden_state=pixel_values)

In [4]:
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"


#model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", device_map="auto", load_in_8bit=True)
#model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", 
# _map="auto", load_in_8bit=True)

model = Blip2ForConditionalGeneration.from_pretrained(
    "ybelkada/blip2-opt-2.7b-fp16-sharded",
    device_map="auto",
    torch_dtype=torch.float32
)

dtype = next(model.parameters()).dtype
print(dtype)

adapted_vit3d = ViT3DAdapter().to(device)
model.vision_model = adapted_vit3d

Loading checkpoint shards:  88%|████████▊ | 7/8 [00:04<00:00,  1.73it/s]

In [ ]:
# Checar se o modelo está congelado
def check_frozen_layers(model):
    frozen_layers = []
    trainable_layers = []

    for name, param in model.named_parameters():
        if param.requires_grad:
            trainable_layers.append(name)
        else:
            frozen_layers.append(name)
    
    print("Camadas congeladas:")
    for layer in frozen_layers:
        print(layer)
    
    print("\nCamadas treináveis:")
    for layer in trainable_layers:
        print(layer)

#check_frozen_layers(model)

Next we define our `LoraConfig` object. We explicitly tell 

In [ ]:
from peft import LoraConfig, get_peft_model
import torch


# Let's define the LoraConfig
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# Congelar todas as camadas do modelo, exceto o Q-Former (apenas q_proj e k_proj)
# for name, param in model.named_parameters():
#     # Mantenha apenas as camadas do Q-Former como treináveis e cheque o tipo de dado
#     if "qformer" in name:
#         #if ('attention' in name or 'crossattention' in name) and torch.is_floating_point(param):
#         param.requires_grad = True
#     else:
#         param.requires_grad = False

# Congelar todas as camadas do modelo, exceto o Q-Former
# for name, param in model.named_parameters():
#     # Checar se o parâmetro faz parte do Q-Former e se é de ponto flutuante
#     if "qformer" in name and torch.is_floating_point(param):
#         param.requires_grad = True
#     else:
#         param.requires_grad = False


# Configurar todos os parâmetros como treináveis, verificando se são de ponto flutuante
# for param in model.parameters():
#     if torch.is_floating_point(param):
#         param.requires_grad = True

# Exibir parâmetros treináveis para verificação
print("Parâmetros treináveis no Q-Former:")
for name, param in model.named_parameters():
    if param.requires_grad:
        # print(name)
        pass
#model.print_trainable_parameters()


In [ ]:
# check_frozen_layers(model)

Now that we have loaded the processor, let's load the dataset and the dataloader:

In [5]:
train_dataset = ImageCaptioningDataset(df_train, embds_path, processor)
test_dataset = ImageCaptioningDataset(df_test, embds_path, processor)
# train_dataset_ = train_dataset_[:100]
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)
#train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4)

In [11]:
print(len(train_dataloader))
print(len(test_dataloader))

103
26


In [ ]:
for i in range(10):
    print(f"Sample {i+1}")    
    d = train_dataset[i]
    print("CT Image embed: ", d['embds'])
    print("Text Report: ", d['text'])
    print("-"*150)

## Train the model

Let's train the model! Run the simply the cell below for training the model

In [ ]:
if False:
    import torch
    import os
    
    epochs = 10
    lr = 5e-4
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Diretório para salvar o melhor modelo
    checkpoint_dir = "./checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    best_model_path = os.path.join(checkpoint_dir, "best_model.pt")
    best_loss = float('inf')  # Define a melhor perda inicial como infinita
    
    for epoch in range(epochs):
        start_time = time()
        train_loss = 0
        total_train_samples = 0
        total_val_samples = 0
        validation_loss = 0
        model.train()
        model.vision_model.training = True
      
        for idx, batch in enumerate(train_dataloader):
            input_ids = batch.pop("input_ids").to(device)
            embds = batch.pop("embds").to(device, torch.float32)
            attention_mask = batch.pop("attention_mask").to(device)
    
            # print(embds.size(), input_ids.size())
            #outputs = model(input_ids=input_ids, pixel_values=embds, labels=input_ids)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=embds, labels=input_ids)
    
            loss = outputs.loss
            # print("Loss:", loss.item())
        
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    
            train_loss += loss.item()
            total_train_samples += len(input_ids)
        
        model.eval()
        validation_loss = 0
        total_val_samples = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for idx, batch in enumerate(test_dataloader):
                input_ids = batch.pop("input_ids").to(device)
                embds = batch.pop("embds").to(device, torch.float32)
                attention_mask = batch.pop("attention_mask").to(device)
                # original_text = batch.pop("original_text")
                model.vision_model.training = True
                # outputs = model(input_ids=input_ids, pixel_values=embds, labels=input_ids)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, pixel_values=embds, labels=input_ids)
    
                loss = outputs.loss
                validation_loss += loss.item()
                total_val_samples += len(input_ids)
                model.vision_model.training = False
                # generated_ids = model.generate(pixel_values=embds, max_length=500)
                # generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
                # original_text = processor.batch_decode(input_ids, skip_special_tokens=True)
                
                # for original, generated in zip(original_text, generated_caption):
                #     print("Original Text -->", original)
                #     print("\n")
                #     print("Generated Text -->", generated)
                #     print("\n")
                # print("\n")
    
         # Calcula a perda média de avaliação
        avg_eval_loss = validation_loss / total_val_samples
        print(f"Validation Loss (Epoch {epoch}): {avg_eval_loss}")
    
        # Salva o modelo se a perda de validação for a melhor até agora
        if avg_eval_loss < best_loss:
            best_loss = avg_eval_loss
            # torch.save({
            #     'epoch': epoch,
            #     'model_state_dict': model.state_dict(),
            #     'optimizer_state_dict': optimizer.state_dict(),
            #     'validation_loss': best_loss,
            # }, best_model_path)
            # print(f"New best model saved with validation loss {best_loss} at {best_model_path}")
            print(f"Saving best model with loss {best_loss}")
            model.save_pretrained(checkpoint_dir)  # Salva o modelo no diretório
            optimizer_state = {"optimizer": optimizer.state_dict(), "epoch": epoch}
            torch.save(optimizer_state, os.path.join(checkpoint_dir, "optimizer.pt"))  # Salva o otimizador
    
    
        end_time = time()  # End time of the epoch
        epoch_duration = end_time - start_time  # Duration of epoch
    
        print(f'Epoch [{epoch+1}/{epochs}], \
                Train Loss: {(train_loss/total_train_samples):.4f}, \
                Val Loss: {(validation_loss/total_val_samples):.4f}, \
                Elapsed Time: {epoch_duration:.2f} sec'
             )

In [ ]:
len(input_ids)

## Inference

Let's check the results on our train dataset

In [7]:
import os
from bert_score import score

device = "cuda" if torch.cuda.is_available() else "cpu"

# Diretório para salvar o melhor modelo
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

model = Blip2ForConditionalGeneration.from_pretrained(
    checkpoint_dir,
    device_map="auto",
    torch_dtype=torch.float32
)

adapted_vit3d = ViT3DAdapter().to(device)
model.vision_model = adapted_vit3d
# Colocar o modelo em modo de avaliação, se necessário

model.eval()

validation_loss = 0
total_val_samples = 0
correct = 0
total = 0

precisions = []
recalls = []
f1s = []

with torch.no_grad():
    for idx, batch in enumerate(test_dataloader):
        input_ids = batch.pop("input_ids").to(device)
        embds = batch.pop("embds").to(device, torch.float32)
        attention_mask = batch.pop("attention_mask").to(device)
        # original_text = batch.pop("original_text")
        model.vision_model.training = True
        # outputs = model(input_ids=input_ids, pixel_values=embds, labels=input_ids)
        outputs = model(input_ids=input_ids, pixel_values=embds, labels=input_ids)

        loss = outputs.loss
        validation_loss += loss.item()
        total_val_samples += len(input_ids)
        model.vision_model.training = False
        generated_ids = model.generate(pixel_values=embds, temperature=0.3, do_sample=True, max_length=500)
        generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
        original_text = processor.batch_decode(input_ids, skip_special_tokens=True)
        
        for original, generated in zip(original_text, generated_caption):
            print("Original Text -->", original)
            print("\n")
            print("Generated Text -->", generated)
            print("\n")
            P, R, F1 = score([generated], [original], lang="en", verbose=True, device=device)
            print("Precision:", P)
            print("Recall:", R)
            print("F1:", F1)

            precisions.append(P)
            recalls.append(R)
            f1s.append(F1)
        
        print("\n")

print("Precision mean: ", torch.mean(precisions))
print("Recall mean: ", torch.mean(recalls))
print("F1 mean: ", torch.mean(f1s))

Loading checkpoint shards: 100%|██████████| 8/8 [00:04<00:00,  1.63it/s]


Original Text --> Trachea and main bronchi are open. No pathological lymph node was detected in the mediastinum. The heart is in natural appearance. The descending thoracic aorta is slightly dilated proximally. Esophagus is within normal limits. Pleural effusion-thickening was not detected in both hemithorax. In the evaluation of both lung parenchyma; A crazy paving appearance consisting of ground glass density and interlobular septal thickening was observed in the superior segment of the left lung lower lobe. Cylindrical bronchiectasis and vascular prominence were observed in this area. Viral pneumonia? An appearance compatible with an intrapulmonary lymph node was observed in the fissure on the left. There are pure calcific millimetric nodules at the left apex. The appearance of millimetric non-specific nodules was observed in the right lung. There is a subsegmental atelectasis appearance in the medial segment of the middle lobe of the right lung. In the sections passing through the 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00, 34.73it/s]


...

100%|██████████| 1/1 [00:00<00:00, 37.07it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 703.39it/s]

done in 0.03 seconds, 32.68 sentences/sec
Precision: tensor([0.8749])
Recall: tensor([0.8538])
F1: tensor([0.8642])




TypeError: mean(): argument 'input' (position 1) must be Tensor, not list

In [8]:

print("Precision mean: ", torch.mean(torch.tensor(precisions)))
print("Recall mean: ", torch.mean(torch.tensor(recalls)))
print("F1 mean: ", torch.mean(torch.tensor(f1s)))

Precision mean:  tensor(0.8836)
Recall mean:  tensor(0.8698)
F1 mean:  tensor(0.8765)
